In [ ]:
import pandas as pd
import plotly.express as px

In [32]:

def clean_data(df):
    # Removendo colunas nulas ou com 100% do mesmo dado
    df = df.drop(columns=['Tags','Público Det.','Natureza','Realização'])

    # Removendo caracteres não numerais
    regex = r"(\d+|[^\d]+)"
    df['Vagas'] = df['Vagas'].str.extract(regex, expand=False)
    df['CH'] = df['CH'].str.extract(regex, expand=False)

    if df['Vagas'].isnull().any():
        df['Vagas'] = df['Vagas'].fillna(value=10)
    
    # Alterando o tipo de dado
    df['Vagas'] = df['Vagas'].astype(int)
    df['CH'] = df['CH'].astype(int)
    
    # Change column type to datetime64[ns] for column: 'Início'
    df['Início'] = pd.to_datetime(df['Início'],format="%d/%m/%Y")
    df['Fim'] = pd.to_datetime(df['Fim'],format="%d/%m/%Y")
    
    # Adicionar uma nova coluna para o ano do projeto pedagógico
    df['Ano do Projeto Pedagógico'] = df['Projeto Pedagógico'].str.extract(regex, expand=False).astype(int)

    # Novas aolunas de dia, mes e ano a partir das colunas início e fim
    df['dia_inicio'] = df['Início'].dt.day
    df['mes_inicio'] = df['Início'].dt.month
    df['ano_inicio'] = df['Início'].dt.year
    
    df['dia_fim'] = df['Fim'].dt.day
    df['mes_fim'] = df['Fim'].dt.month
    df['ano_fim'] = df['Fim'].dt.year
    
    return df

# Loaded variable 'df' from URI: /workspaces/python/dataset/formacoes-por-periodo.csv
# df = pd.read_csv(r'C:\\Documentos\\projetos-git\\jupyter-notebook\\dataset\\planilha-08-05-2024.csv')
df = pd.read_excel(r'C:\\Documentos\\projetos-git\\jupyter-notebook\\dataset\\2024-05-09-emeronweb.xls')

df_clean = clean_data(df.copy())

In [33]:
df_clean.columns

Index(['Unnamed: 0', 'Situação', 'Área Temática', 'Modalidade',
       'Tipo de Atividade', 'Data', 'Atividade', 'Vagas', 'N. Alunos',
       'Concluintes', 'Não Concluintes', 'CH', 'Início', 'Fim', 'Local',
       'Público', 'Setor', 'Projeto Pedagógico', 'Ano do Projeto Pedagógico',
       'dia_inicio', 'mes_inicio', 'ano_inicio', 'dia_fim', 'mes_fim',
       'ano_fim'],
      dtype='object')

In [ ]:
df_clean.info()

- Cursos por público

In [ ]:
df_clean['Público'].value_counts()

- Crio um dataframe para trabalhar especificamente as métricas

In [ ]:
df_metricas = df_clean[['Início','Atividade','Vagas','N. Alunos','Concluintes','Não Concluintes','CH']]

df_metricas.head(3)

In [ ]:
df_metricas.drop(columns='CH').value_counts()

## Numero de vagas, alunos, concluintes, carga horária

### Métricas:
- Alunos por vagas (total anual)
- Concluintes por não concluintes (total anual)
- Vagas abertas por alunos inscritos no mês
- Concluintes por não concluintes (por mês de encerramento do curso)
- Total de atividades por mês

In [ ]:
df_vagas_alunos_concluintes = df_metricas.drop(columns=['Início', 'Atividade','CH', 'Não Concluintes']).sum()
df_vagas_alunos_concluintes

- Carga horária total

In [ ]:
df_metricas['CH'].sum()

- Total de Atividaes

In [ ]:
df_metricas['Atividade'].count()

- Total de atividades por mês

In [ ]:
df_clean.groupby("mes_inicio")["Atividade"].count()

- Total de vagas abertas por mês

In [ ]:
df_clean.groupby('mes_inicio')['Vagas'].sum()

- Mes que mais encerrou cursos por concluientes

In [ ]:
df_clean.groupby('mes_fim')['Concluintes'].sum()

In [ ]:
### Filtro Aguardando: Vagas, Alunos, Concluintes e Não Concluintes, exceto mês 12
df_filtroAguardando = df_clean[['Situação','mes_inicio','mes_fim','Vagas','N. Alunos','Concluintes','Não Concluintes']]

df_filtroAguardando = df_filtroAguardando[(df_clean['Situação'] == 'Aguardando') & (df_clean['mes_fim'] < 12)]

df_filtroAguardando.sample(3)

In [ ]:
bimestre = df_clean[['Início','Vagas']].groupby(pd.Grouper(key='Início', freq='2MS')).sum()
bimestre

In [ ]:
trimestre = df_clean[['Início','Vagas']].groupby(pd.Grouper(key='Início', freq='3MS')).sum()
trimestre

In [ ]:
quadrimestre = df_clean[['Início','Vagas']].groupby(pd.Grouper(key='Início', freq='4MS')).sum()
quadrimestre

In [ ]:
semestre = df_clean[['Início','Vagas']].groupby(pd.Grouper(key='Início', freq='6MS')).sum()
semestre

# Gráficos

- Número cursos executados por número de cursos aguardando

In [ ]:
df_clean.columns

In [ ]:
df_clean['Situação'].value_counts().reset_index()